In [2]:
%pip install pymongo

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
import numpy as np
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo.errors import OperationFailure
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import os
import io
import pickle

In [5]:
# Fazendo dataset para um sistema de ocorrências de bombeiros
N = 2000
df = pd.DataFrame({
    'ID': range(N),
    'Grupo de Ocorrência': np.random.choice(['Incêndio', 'Resgate', 'Acidente de Trânsito', 'Desabamento'], N),
    'Tipo de Ocorrência': np.random.choice(['Residencial', 'Comercial', 'Industrial', 'Público'], N),
    'Prioridade': np.random.choice(['Baixa', 'Média', 'Alta', 'Crítica'], N),
    'Tempo de Resposta (minutos)': np.random.randint(5, 60, N),
    'Número de Vítimas': np.random.randint(0, 10, N),
    'Danos Estimados (R$)': np.random.randint(1000, 50000, N)
})
df

,ID,Grupo de Ocorrência,Tipo de Ocorrência,Prioridade,Tempo de Resposta (minutos),Número de Vítimas,Danos Estimados (R$)
0,0,Desabamento,Comercial,Baixa,16,6,2547
1,1,Acidente de Trânsito,Residencial,Média,41,4,2731
2,2,Acidente de Trânsito,Comercial,Média,43,7,24669
3,3,Desabamento,Residencial,Crítica,18,2,3292
4,4,Acidente de Trânsito,Industrial,Baixa,25,2,40390
...,...,...,...,...,...,...,...
1995,1995,Acidente de Trânsito,Público,Alta,15,7,8617
1996,1996,Incêndio,Público,Alta,50,4,48343
1997,1997,Desabamento,Comercial,Baixa,35,8,32729
1998,1998,Acidente de Trânsito,Comercial,Baixa,23,1,3512


In [6]:
#df to json
df.to_json('df.json', orient='records', lines=True)

In [ ]:
# MongoDB Connection
MONGO_URI = os.environ.get('MONGO_URI', 'mongodb://localhost:27017/')
client = MongoClient(MONGO_URI)
db = client["chama_sos"]
collection = db["chama_sos"]

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Conectado ao MongoDB!")
except Exception as e:
    print('MongoDB ping failed:', type(e).__name__, e)
    print('If youre using Atlas, ensure your connection string includes credentials and your IP is whitelisted.')
    raise

with open('df.json', 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue
        # Use io.StringIO(line) to pass a file-like object instead of a literal string
        record = pd.read_json(io.StringIO(line), typ='series') 
        # convert numpy scalars to native Python types to avoid BSON type issues
        doc = record.apply(lambda x: x.item() if hasattr(x, "item") else x).to_dict()
        try:
            collection.insert_one(doc)
        except OperationFailure as e:
            print('Inserção falhou com OperationFailure:', e)
            print('Check credentials/permissions, and ensure the user has insert privileges. If using Atlas, verify the connection string and IP access list.')
            raise

print("Inserção de dados completa!")